## 把网页保存到es中

In [1]:
import pymysql
# 创建连接
cnx = pymysql.connect(host='localhost', user='root', password='123456')
cursor = cnx.cursor()

In [2]:
ids_douban = []
ids_page = []

def create_database():
    # 创建数据库
    cursor.execute("CREATE DATABASE IF NOT EXISTS IR_db")
    # 使用数据库
    cnx.select_db('IR_db')

def get_ids_from_douban():
    # 创建SQL查询
    sql = "SELECT id FROM douban"
    # 执行查询
    cursor.execute(sql)
    # 获取所有的结果
    results = cursor.fetchall()
    # 将结果从元组列表转换为普通列表
    ids_douban = [result[0] for result in results]
    return ids_douban

def get_ids_from_same_title():
    # 创建SQL查询
    sql = "SELECT page_id FROM same_title"
    # 执行查询
    cursor.execute(sql)
    # 获取所有的结果
    results = cursor.fetchall()
    # 将结果从元组列表转换为普通列表
    ids_same_title = [result[0] for result in results]
    return ids_same_title

def get_ids_from_page():
    global ids_page
    # 创建SQL查询
    sql = "SELECT id FROM page"
    # 执行查询
    cursor.execute(sql)
    # 获取所有的结果
    results = cursor.fetchall()
    # 将结果从元组列表转换为普通列表
    ids = [result[0] for result in results]
    # 获取相同title的id
    ids_same_title = get_ids_from_same_title()
    print(len(ids_same_title))
    # 去除相同title的id
    for id in ids:
        if id not in ids_same_title:
            ids_page.append(id)
    # 从same_title中按title分组，获取每组的第一个id
    sql = "SELECT MIN(page_id) FROM same_title GROUP BY title"
    cursor.execute(sql)
    results = cursor.fetchall()
    ids = [result[0] for result in results]
    # 将结果添加到ids_page中
    ids_page.extend(ids)

def get_content(table, id):
    # 创建SQL查询
    sql = "SELECT * FROM {} WHERE id = %s".format(table)
    # 执行查询
    cursor.execute(sql, (id,))
    # 获取结果
    result = cursor.fetchone()
    # 如果结果不为空，将结果转换为字典并返回，否则返回None
    if result:
        return {
            'id': result[0],
            'title': result[1],
            'content': result[2],
            'url': result[3]
        }
    else:
        return None

create_database()
ids_douban = get_ids_from_douban()
print(len(ids_douban))
get_ids_from_page()
print(len(ids_page))


2189
7202
17460


### 创建es索引

In [3]:
from elasticsearch import Elasticsearch

# 实例化es
es = Elasticsearch(hosts="http://localhost:9200")
#es.indices.create(index='web')
# 定义索引设置和映射
body = {
    "settings": {
        "analysis": {
            "analyzer": {
                "default": {
                    "type": "ik_max_word"
                }
            }
        }
    },
    "mappings": {
        "properties": {
            "web_id": {
                "type": "integer"
            },
            "title": {
                "type": "text",
                "analyzer": "ik_max_word"
            },
            "content": {
                "type": "text",
                "analyzer": "ik_max_word"
            },
            "url": {
                "type": "keyword"
            },
            "type": {
                "type": "keyword"
            }
        }
    }
}

# 创建索引
es.indices.create(index='web', body=body, ignore = 400)

C:\Users\89214\AppData\Local\Temp\ipykernel_28252\2980139286.py:41: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.
  es.indices.create(index='web', body=body, ignore = 400)
C:\Users\89214\AppData\Local\Temp\ipykernel_28252\2980139286.py:41: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  es.indices.create(index='web', body=body, ignore = 400)


ObjectApiResponse({'error': {'root_cause': [{'type': 'resource_already_exists_exception', 'reason': 'index [web/MV6eVvvyQNWjDr_AYbbVkw] already exists', 'index_uuid': 'MV6eVvvyQNWjDr_AYbbVkw', 'index': 'web'}], 'type': 'resource_already_exists_exception', 'reason': 'index [web/MV6eVvvyQNWjDr_AYbbVkw] already exists', 'index_uuid': 'MV6eVvvyQNWjDr_AYbbVkw', 'index': 'web'}, 'status': 400})

### 读取数据插入web

In [4]:
global_id = 1
def insert_douban_web():
    global global_id
    for id in ids_douban:
        # 获取内容
        content = get_content('douban', id)
        doc = {
            'web_id': content['id'],
            'title': content['title'],
            'content': content['content'],
            'url': content['url'],
            'type': 'douban'
        }
        """
        # 插入数据
        es.create(index='web', id = global_id, document=doc)
        global_id += 1
        """
        if not es.exists(index='web', id=global_id):
            es.create(index='web', id = global_id, document=doc)
        else:
            print("文档 {global_id} 已存在，跳过创建")
        global_id += 1

def insert_page_web():
    global global_id
    for id in ids_page:
        # 获取内容
        content = get_content('page', id)
        doc = {
            'web_id': content['id'],
            'title': content['title'],
            'content': content['content'],
            'url': content['url'],
            'type': 'page'
        }
        """
        # 插入数据
        es.create(index='web', id = global_id, document=doc)
        global_id += 1    
        """
        if not es.exists(index='web', id = global_id):
            es.create(index='web', id = global_id, document=doc)
        else:
            print(f"文档 {global_id} 已存在，跳过创建")
        global_id += 1

In [5]:
global_id = 1
insert_douban_web()
print('douban done', global_id)
insert_page_web()
print('page done', global_id)

C:\Users\89214\AppData\Local\Temp\ipykernel_28252\2726686180.py:19: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  if not es.exists(index='web', id=global_id):


文档 {global_id} 已存在，跳过创建
文档 {global_id} 已存在，跳过创建
文档 {global_id} 已存在，跳过创建
文档 {global_id} 已存在，跳过创建
文档 {global_id} 已存在，跳过创建
文档 {global_id} 已存在，跳过创建
文档 {global_id} 已存在，跳过创建
文档 {global_id} 已存在，跳过创建
文档 {global_id} 已存在，跳过创建
文档 {global_id} 已存在，跳过创建
文档 {global_id} 已存在，跳过创建
文档 {global_id} 已存在，跳过创建
文档 {global_id} 已存在，跳过创建
文档 {global_id} 已存在，跳过创建
文档 {global_id} 已存在，跳过创建
文档 {global_id} 已存在，跳过创建
文档 {global_id} 已存在，跳过创建
文档 {global_id} 已存在，跳过创建
文档 {global_id} 已存在，跳过创建
文档 {global_id} 已存在，跳过创建
文档 {global_id} 已存在，跳过创建
文档 {global_id} 已存在，跳过创建
文档 {global_id} 已存在，跳过创建
文档 {global_id} 已存在，跳过创建
文档 {global_id} 已存在，跳过创建
文档 {global_id} 已存在，跳过创建
文档 {global_id} 已存在，跳过创建
文档 {global_id} 已存在，跳过创建
文档 {global_id} 已存在，跳过创建
文档 {global_id} 已存在，跳过创建
文档 {global_id} 已存在，跳过创建
文档 {global_id} 已存在，跳过创建
文档 {global_id} 已存在，跳过创建
文档 {global_id} 已存在，跳过创建
文档 {global_id} 已存在，跳过创建
文档 {global_id} 已存在，跳过创建
文档 {global_id} 已存在，跳过创建
文档 {global_id} 已存在，跳过创建
文档 {global_id} 已存在，跳过创建
文档 {global_id} 已存在，跳过创建
文档 {global_id} 已存在，跳过创建
文档 {global_id} 已

C:\Users\89214\AppData\Local\Temp\ipykernel_28252\2726686180.py:42: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  if not es.exists(index='web', id = global_id):


文档 2198 已存在，跳过创建
文档 2199 已存在，跳过创建
文档 2200 已存在，跳过创建
文档 2201 已存在，跳过创建
文档 2202 已存在，跳过创建
文档 2203 已存在，跳过创建
文档 2204 已存在，跳过创建
文档 2205 已存在，跳过创建
文档 2206 已存在，跳过创建
文档 2207 已存在，跳过创建
文档 2208 已存在，跳过创建
文档 2209 已存在，跳过创建
文档 2210 已存在，跳过创建
文档 2211 已存在，跳过创建
文档 2212 已存在，跳过创建
文档 2213 已存在，跳过创建
文档 2214 已存在，跳过创建
文档 2215 已存在，跳过创建
文档 2216 已存在，跳过创建
文档 2217 已存在，跳过创建
文档 2218 已存在，跳过创建
文档 2219 已存在，跳过创建
文档 2220 已存在，跳过创建
文档 2221 已存在，跳过创建
文档 2222 已存在，跳过创建
文档 2223 已存在，跳过创建
文档 2224 已存在，跳过创建
文档 2225 已存在，跳过创建
文档 2226 已存在，跳过创建
文档 2227 已存在，跳过创建
文档 2228 已存在，跳过创建
文档 2229 已存在，跳过创建
文档 2230 已存在，跳过创建
文档 2231 已存在，跳过创建
文档 2232 已存在，跳过创建
文档 2233 已存在，跳过创建
文档 2234 已存在，跳过创建
文档 2235 已存在，跳过创建
文档 2236 已存在，跳过创建
文档 2237 已存在，跳过创建
文档 2238 已存在，跳过创建
文档 2239 已存在，跳过创建
文档 2240 已存在，跳过创建
文档 2241 已存在，跳过创建
文档 2242 已存在，跳过创建
文档 2243 已存在，跳过创建
文档 2244 已存在，跳过创建
文档 2245 已存在，跳过创建
文档 2246 已存在，跳过创建
文档 2247 已存在，跳过创建
文档 2248 已存在，跳过创建
文档 2249 已存在，跳过创建
文档 2250 已存在，跳过创建
文档 2251 已存在，跳过创建
文档 2252 已存在，跳过创建
文档 2253 已存在，跳过创建
文档 2254 已存在，跳过创建
文档 2255 已存在，跳过创建
文档 2256 已存在，跳过

In [6]:
# es.indices.delete(index='web')

### 测试查询数据

In [7]:
response = es.count(index='web')
print('Document count:', response['count'])

Document count: 19649


C:\Users\89214\AppData\Local\Temp\ipykernel_28252\468739004.py:1: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  response = es.count(index='web')


In [8]:
def deal_bool_query(query):
    # 执行搜索
    response = es.search(
        index="web",
        body={
            "query": {
                "bool": {
                    "must": [
                        {
                            "multi_match": {
                                "query": query,
                                "fields": ["title", "content"]
                            }
                        }
                    ]
                }
            }
        }
    )
    # 获取搜索结果
    hits = response["hits"]["hits"]
    # 格式化搜索结果
    results = [
        {
            "id": hit["_id"],
            "title": hit["_source"]["title"],
            "content": hit["_source"]["content"],
            "url": hit["_source"]["url"],
            "type": hit["_source"]["type"]
        }
        for hit in hits
    ]
    return results

In [9]:
results = deal_bool_query('肖申克的救赎')
for result in results:
    print(result['title'], result['url'], result['type'])

肖申克的救赎 https://book.douban.com/subject/1829226/ douban
汲汲而生，汲汲而死 https://book.douban.com/review/1597365/ douban
肖申克的救赎，读书笔记 https://book.douban.com/review/1336253/ douban
父亲与救赎——《基督山伯爵》《肖申克救赎》心理分析 https://book.douban.com/review/3515997/ douban
我们都有一颗邪恶的心 https://book.douban.com/review/1202075/ douban
再强大的心理也会崩溃——读《纳粹高徒》有感 https://book.douban.com/review/5582004/ douban
书和电影的不同 https://book.douban.com/review/1076780/ douban
等待和希望 https://book.douban.com/review/1202904/ douban
hope is a good thing https://book.douban.com/review/1379728/ douban
时刻准备着对这个一无是处的世界失望 https://book.douban.com/review/7733801/ douban


C:\Users\89214\AppData\Local\Temp\ipykernel_28252\3382472245.py:3: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  response = es.search(
